In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
import pywt
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from skimage.transform import resize
import os
from scipy.io import wavfile
from scipy.io.wavfile import read, write
from scipy.signal import resample_poly
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Functions

In [2]:
def signal_resample(signal, original_fs, target_fs):
    gcd = np.gcd(original_fs, target_fs)
    up = target_fs // gcd
    down = original_fs // gcd
    resampled_signal=resample_poly(signal, up, down)
    return resampled_signal
    
def create_scalogram(sig,fs):
    scales = np.arange(1, 41)
    
    coeffs, _ = pywt.cwt(sig, scales, wavelet = 'morl', sampling_period=1/fs)
    f = abs(coeffs)
    f = resize(f, (40, 250), mode = 'constant')
    return f

def normalize(array):
    return (array - array.min())/(array.max() - array.min()) 

# mimic talking

In [8]:
fs=4000
tensor_rec=np.zeros((1,250,40,1))

original_fs, original_recording = wavfile.read('mimic_talking.wav')


DUS_Resampled = signal_resample(original_recording.astype('float32'), original_fs, fs) #Resampling the singal to 4 kHz
DUS_Normalized=normalize(DUS_Resampled) #Normalizing the singal
f=create_scalogram(DUS_Normalized,fs) #Creating the scalogram
f=normalize(f) #Normalizing the scalogram
tensor_rec[0,:,:,0]=f.T 

model_path = './saved_model'
model = tf.keras.models.load_model(model_path)  # loading the saved model

label = np.argmax(model.predict(tensor_rec), axis=1) 

quality_labels = {0: "Good", 1: "Poor", 2: "Interference", 3: "Talking", 4: "Silent"}

# Print the quality labels
Estimated_label = quality_labels[label[0]]
print(f'The Estimated Quality Class is "{Estimated_label}"')

1/1 [==============================] - 0s 184ms/step
The Estimated Quality Class is "Talking"


# Mimic talking on "Good"

In [9]:
fs=4000
tensor_rec=np.zeros((1,250,40,1))

original_fs, original_recording = wavfile.read('mimic_talking_on_good.wav')


DUS_Resampled = signal_resample(original_recording.astype('float32'), original_fs, fs) #Resampling the singal to 4 kHz
DUS_Normalized=normalize(DUS_Resampled) #Normalizing the singal
f=create_scalogram(DUS_Normalized,fs) #Creating the scalogram
f=normalize(f) #Normalizing the scalogram
tensor_rec[0,:,:,0]=f.T 

model_path = './saved_model'
model = tf.keras.models.load_model(model_path)  # loading the saved model

label = np.argmax(model.predict(tensor_rec), axis=1) 

quality_labels = {0: "Good", 1: "Poor", 2: "Interference", 3: "Talking", 4: "Silent"}

# Print the quality labels
Estimated_label = quality_labels[label[0]]
print(f'The Estimated Quality Class is "{Estimated_label}"')

1/1 [==============================] - 0s 178ms/step
The Estimated Quality Class is "Interference"
